In [12]:
####################
## load libraries ##
####################
import numpy as np
import pandas as pd
np.random.seed(123456789)
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm

In [13]:
##############################################
## Function for simulating Regression Model ##
##############################################
def simulate_df(n=1000, glm_type='logistic', seed=123456):
    np.random.seed(seed)
    df = pd.DataFrame()
    ## specify variables L1 through L6
    L1_split = 0.52
    L2_split = 0.23
    L3_split = 0.38
    df['L0'] = n*[1]
    df['L1'] = np.random.choice([0, 1], size=n, replace=True, p=[L1_split, (1-L1_split)])
    df['L2'] = np.random.choice([0, 1], size=n, replace=True, p=[L2_split, (1-L2_split)])
    df['L3'] = np.random.choice([0, 1], size=n, replace=True, p=[L3_split, (1-L3_split)])
    df['L4'] = np.random.normal(0, 1, df.shape[0])
    df['L5'] = np.random.normal(0, 0.75, df.shape[0])
    df['L6'] = np.random.normal(0, 2, df.shape[0])
    df['L4'] = df['L4'] / max(abs(df['L4']))
    df['L5'] = df['L5'] / max(abs(df['L5']))
    df['L6'] = df['L6'] / max(abs(df['L6']))
    
    ## define beta parameters
    beta_0 = -0.3
    beta_1 = -1.58
    beta_2 = 1.75
    beta_3 = 0.42
    beta_4 = 1.32
    beta_5 = -1.15
    beta_6 = 1.12

    if(glm_type=='logistic'):
        df['Z'] = beta_0 + (beta_1*df['L1']) + (beta_2*df['L2']) + (beta_3*df['L3']) + (beta_4*df['L4']) + (beta_5*df['L5']) + (beta_6*df['L6'])
        df['p'] = 1 / (1 + np.exp(-df['Z']))
        df['Y'] = np.random.binomial(1, df['p'])
        model = smf.glm('Y ~ L1 + L2 + L3 + L4 + L5 + L6', data=df, family=sm.families.Binomial()).fit()
    
    if(glm_type=='poisson'):
        df['Z'] = beta_0 + (beta_1*df['L1']) + (beta_2*df['L2']) + (beta_3*df['L3']) + (beta_4*df['L4']) + (beta_5*df['L5']) + (beta_6*df['L6'])
        df['p'] = np.exp(df['Z'])
        df['Y'] = None
        for i in range(0, df.shape[0]):
            df.loc[i, 'Y'] = np.random.poisson(df.loc[i, 'p'], 1)
        df['Y'] = df['Y'].astype(float)
        model = smf.glm('Y ~ L1 + L2 + L3 + L4 + L5 + L6', data=df, family=sm.families.Poisson()).fit()
    
    if(glm_type=='probit'):
        df['Z'] = beta_0 + (beta_1*df['L1']) + (beta_2*df['L2']) + (beta_3*df['L3']) + (beta_4*df['L4']) + (beta_5*df['L5']) + (beta_6*df['L6'])
        df['p'] = norm.cdf(df['Z'])
        df['Y'] = np.random.binomial(1, df['p'])
        model = statsmodels.discrete.discrete_model.Probit(df['Y'], df[['L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']]).fit()
    
    return(df, model)

In [14]:
##########################
## Activation Functions ##
##########################
def sigmoid_activation(Z):
    p = 1 / (1 + np.exp(-Z))
    return(p)

def exponential_activation(Z):
    p = np.exp(Z)
    return(p)
    
def probit_activation(Z):
    p = norm.cdf(Z)
    return(p)

In [15]:
#######################################################################
## Function for Newton-Raphson and Fisher Scoring for Canonical GLMs ##
#######################################################################
def NewtonRaphson_FisherScoring_Canonical(df, glm_type='logistic', seed=123456):

    np.random.seed(seed)
    ## specify number of observations and number of features (including intercept)
    n = df.shape[0]
    m = 7
    if(glm_type=='logistic'):
        k = 10
    if(glm_type=='poisson'):
        k = 100
        
    ## specify the Target Y
    Y = np.array(df['Y']).reshape((n,1))
    ## specify design matrix X        
    X = np.array(df[['L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']])
    ## initialize Beta vector:
    Beta = np.random.normal(0, 0.5, m).reshape((m, 1))
        
    ## calculate the initial cost J
    if(glm_type=='logistic'):
        b_theta = np.log(1+np.exp(np.dot(X, Beta)))
    if(glm_type=='poisson'):
        b_theta = np.exp(np.dot(X, Beta))
    J = np.sum(b_theta - (np.dot(X, Beta) * Y))
        
    ## dataframe to hold algo results
    df_algo_results = pd.DataFrame()
    df_algo_results['k'] = list(range(0, k+1))
    df_algo_results['J'] = None
    df_algo_results.loc[0, 'J'] = J
    del b_theta, J
        
    for i in range(0, k):    
        ## get linear predictor Z
        Z = np.dot(X, Beta)
        ## get prediction p
        if(glm_type=='logistic'):
            p = sigmoid_activation(Z)
        if(glm_type=='poisson'):
            p = exponential_activation(Z)
        ## get variance matrix V
        V = np.diag(((p-Y)**2).squeeze(axis=1))

        ## calculate the Hessian
        Hessian = np.linalg.solve(np.dot(X.T, V).dot(X), np.eye(m))
        
        ## calculate dJ_Beta
        dJ_Beta = np.dot(X.T, (p-Y))
        
        ## get the updated Beta
        Beta = Beta - np.dot(Hessian, dJ_Beta)
            
        ## record cost function
        if(glm_type=='logistic'):
            b_theta = np.log(1+np.exp(np.dot(X, Beta)))
        if(glm_type=='poisson'):
            b_theta = np.exp(np.dot(X, Beta))
        J = np.sum(b_theta - (np.dot(X, Beta) * Y))
        df_algo_results.loc[i+1, 'J'] = J
            
        del J, b_theta, Z, p, V, Hessian, dJ_Beta
    
    return(Beta, df_algo_results)

In [16]:
##########################################
## Function for IRLS for Canonical GLMs ##
##########################################
def IRLS_Canonical(df, glm_type='logistic', seed=123456):

    np.random.seed(seed)
    ## specify number of observations and number of features (including intercept)
    n = df.shape[0]
    m = 7
    if(glm_type=='logistic'):
        k = 10
    if(glm_type=='poisson'):
        k = 100
        
    ## specify the Target Y
    Y = np.array(df['Y']).reshape((n,1))
    ## specify design matrix X        
    X = np.array(df[['L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']])
    ## initialize Beta vector:
    Beta = np.random.normal(0, 0.5, m).reshape((m, 1))
        
    ## calculate the initial cost J
    if(glm_type=='logistic'):
        b_theta = np.log(1+np.exp(np.dot(X, Beta)))
    if(glm_type=='poisson'):
        b_theta = np.exp(np.dot(X, Beta))
    J = np.sum(b_theta - (np.dot(X, Beta) * Y))
        
    ## dataframe to hold algo results
    df_algo_results = pd.DataFrame()
    df_algo_results['k'] = list(range(0, k+1))
    df_algo_results['J'] = None
    df_algo_results.loc[0, 'J'] = J
    del b_theta, J
        
    for i in range(0, k):    
        ## get linear predictor Z
        Z = np.dot(X, Beta)
        ## get prediction p
        if(glm_type=='logistic'):
            p = sigmoid_activation(Z)
        if(glm_type=='poisson'):
            p = exponential_activation(Z)
        ## get variance matrix V
        V = np.diag(((p-Y)**2).squeeze(axis=1))
        
        ## get matrix W_inv and Y_til
        W_inv = V
        Y_til = 1 / (p-Y)
            
        ## calculate WLS estimator
        WLS = np.linalg.solve(np.dot(X.T, W_inv).dot(X), np.eye(m)).dot(X.T).dot(W_inv).dot(Y_til)            
            
        ## get the updated Beta
        Beta = Beta - WLS
            
        ## record cost function
        if(glm_type=='logistic'):
            b_theta = np.log(1+np.exp(np.dot(X, Beta)))
        if(glm_type=='poisson'):
            b_theta = np.exp(np.dot(X, Beta))
        J = np.sum(b_theta - (np.dot(X, Beta) * Y))
        df_algo_results.loc[i+1, 'J'] = J
            
        del J, b_theta, Z, p, V, W_inv, Y_til, WLS
    
    return(Beta, df_algo_results)

In [17]:
########################################################
## Function for Newton-Raphson for Non-Canonical GLMs ##
########################################################
def NewtonRaphson_NonCanonical(df, seed=123456):

    np.random.seed(seed)
    ## specify number of observations and number of features (including intercept)
    n = df.shape[0]
    m = 7
    k = 20
        
    ## specify the Target Y
    Y = np.array(df['Y']).reshape((n,1))
    ## specify design matrix X        
    X = np.array(df[['L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']])
    ## initialize Beta vector:
    Beta = np.random.normal(0, 0.5, m).reshape((m, 1))
        
    ## calculate the initial cost J
    T_y = Y
    Eta = np.dot(X, Beta)
    h_eta = probit_activation(Eta)
    xi = h_eta
    r_xi = np.log(xi / (1-xi)) 
    b_xi = -np.log(1 - xi)
    J = np.sum(b_xi - (T_y * r_xi))
    
    ## dataframe to hold algo results
    df_algo_results = pd.DataFrame()
    df_algo_results['k'] = list(range(0, k+1))
    df_algo_results['J'] = None
    df_algo_results.loc[0, 'J'] = J
    del Eta, h_eta, xi, b_xi, r_xi, J
               
    for i in range(0, k):    
        ## get linear predictor Eta
        Eta = np.dot(X, Beta)
        
        ## get activation function h_eta
        h_eta = probit_activation(Eta)
        ## get first derivative of h_eta with respect to eta
        dh_deta = norm.pdf(Eta)
        
        ## get prediction xi
        xi = h_eta
        
        ## get function r_xi
        r_xi = np.log(xi / (1-xi)) 
        ## get first derivative of r_xi with respect to xi
        dr_dxi = 1 / (xi-(xi**2))
        ## get second derivative of r_xi with respect to xi
        d2r_d2xi = ((2*xi)-1) / ((xi-(xi**2))**2)
        
        ## get function b_xi
        b_xi = -np.log(1 - xi)
        ## get first derivative of b_xi with respect to xi
        db_dxi = 1 / (1-xi)
        ## get second derivative of b_xi with respect to xi
        d2b_d2xi = (2-xi) / ((1-xi)**2)
        
        ## calculate the Hessian
        Hessian = np.linalg.solve(np.dot(X.T, np.diag(((d2b_d2xi - (T_y*d2r_d2xi))*(dh_deta**2)).squeeze(axis=1))).dot(X), np.eye(m))                                          
        
        ## calculate dJ_Beta
        dJ_Beta = np.dot(X.T, ((db_dxi - (T_y*dr_dxi)) * dh_deta))
        
        ## get the updated Beta
        Beta = Beta - np.dot(Hessian, dJ_Beta)
        del Eta, h_eta, dh_deta, xi, r_xi, dr_dxi, d2r_d2xi, b_xi, db_dxi, d2b_d2xi, Hessian, dJ_Beta
        
        ## calculate the updated cost J
        Eta = np.dot(X, Beta)
        h_eta = probit_activation(Eta)
        xi = h_eta
        r_xi = np.log(xi / (1-xi)) 
        b_xi = -np.log(1 - xi)
        J = np.sum(b_xi - (T_y * r_xi))
        df_algo_results.loc[i+1, 'J'] = J
        del Eta, h_eta, xi, b_xi, r_xi, J
    
    return(Beta, df_algo_results)

In [18]:
########################################################
## Function for Fisher Scoring for Non-Canonical GLMs ##
########################################################
def FisherScoring_NonCanonical(df, seed=123456):

    np.random.seed(seed)
    ## specify number of observations and number of features (including intercept)
    n = df.shape[0]
    m = 7
    k = 10
        
    ## specify the Target Y
    Y = np.array(df['Y']).reshape((n,1))
    ## specify design matrix X        
    X = np.array(df[['L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']])
    ## initialize Beta vector:
    Beta = np.random.normal(0, 0.5, m).reshape((m, 1))
        
    ## calculate the initial cost J
    T_y = Y
    Eta = np.dot(X, Beta)
    h_eta = probit_activation(Eta)
    xi = h_eta
    r_xi = np.log(xi / (1-xi)) 
    b_xi = -np.log(1 - xi)
    J = np.sum(b_xi - (T_y * r_xi))    
    
    ## dataframe to hold algo results
    df_algo_results = pd.DataFrame()
    df_algo_results['k'] = list(range(0, k+1))
    df_algo_results['J'] = None
    df_algo_results.loc[0, 'J'] = J
    del Eta, h_eta, xi, b_xi, r_xi, J
               
    for i in range(0, k):    
        ## get linear predictor Eta
        Eta = np.dot(X, Beta)
        
        ## get activation function h_eta
        h_eta = probit_activation(Eta)
        ## get first derivative of h_eta with respect to eta
        dh_deta = norm.pdf(Eta)
        
        ## get prediction xi
        xi = h_eta
        
        ## get function r_xi
        r_xi = np.log(xi / (1-xi)) 
        ## get first derivative of r_xi with respect to xi
        dr_dxi = 1 / (xi-(xi**2))
        ## get second derivative of r_xi with respect to xi
        d2r_d2xi = ((2*xi)-1) / ((xi-(xi**2))**2)
        
        ## get function b_xi
        b_xi = -np.log(1 - xi)
        ## get first derivative of b_xi with respect to xi
        db_dxi = 1 / (1-xi)
        ## get second derivative of b_xi with respect to xi
        d2b_d2xi = (2-xi) / ((1-xi)**2)
        
        ## calculate the Fisher Information
        FisherInformation = np.linalg.solve(np.dot(X.T, np.diag(((dr_dxi**2)*(dh_deta**2)*((xi-T_y)**2)).squeeze(axis=1))).dot(X), np.eye(m)) 

        ## calculate dJ_Beta
        dJ_Beta = np.dot(X.T, ((db_dxi - (T_y*dr_dxi)) * dh_deta))
        
        ## get the updated Beta
        Beta = Beta - np.dot(FisherInformation, dJ_Beta)
        del Eta, h_eta, dh_deta, xi, r_xi, dr_dxi, d2r_d2xi, b_xi, db_dxi, d2b_d2xi, FisherInformation, dJ_Beta
        
        ## calculate the updated cost J
        Eta = np.dot(X, Beta)
        h_eta = probit_activation(Eta)
        xi = h_eta
        r_xi = np.log(xi / (1-xi)) 
        b_xi = -np.log(1 - xi)
        J = np.sum(b_xi - (T_y * r_xi))
        df_algo_results.loc[i+1, 'J'] = J
        del Eta, h_eta, xi, b_xi, r_xi, J
    
    return(Beta, df_algo_results)

In [19]:
##############################################
## Function for IRLS for Non-Canonical GLMs ##
##############################################
def IRLS_NonCanonical(df, seed=123456):

    np.random.seed(seed)
    ## specify number of observations and number of features (including intercept)
    n = df.shape[0]
    m = 7
    k = 10
        
    ## specify the Target Y
    Y = np.array(df['Y']).reshape((n,1))
    ## specify design matrix X        
    X = np.array(df[['L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']])
    ## initialize Beta vector:
    Beta = np.random.normal(0, 0.5, m).reshape((m, 1))
        
    ## calculate the initial cost J
    T_y = Y
    Eta = np.dot(X, Beta)
    h_eta = probit_activation(Eta)
    xi = h_eta
    r_xi = np.log(xi / (1-xi)) 
    b_xi = -np.log(1 - xi)
    J = np.sum(b_xi - (T_y * r_xi))    
    
    ## dataframe to hold algo results
    df_algo_results = pd.DataFrame()
    df_algo_results['k'] = list(range(0, k+1))
    df_algo_results['J'] = None
    df_algo_results.loc[0, 'J'] = J
    del Eta, h_eta, xi, b_xi, r_xi, J
               
    for i in range(0, k):    
        ## get linear predictor Eta
        Eta = np.dot(X, Beta)
        
        ## get activation function h_eta
        h_eta = probit_activation(Eta)
        ## get first derivative of h_eta with respect to eta
        dh_deta = norm.pdf(Eta)
        
        ## get prediction xi
        xi = h_eta
        
        ## get function r_xi
        r_xi = np.log(xi / (1-xi)) 
        ## get first derivative of r_xi with respect to xi
        dr_dxi = 1 / (xi-(xi**2))
        ## get second derivative of r_xi with respect to xi
        d2r_d2xi = ((2*xi)-1) / ((xi-(xi**2))**2)
        
        ## get function b_xi
        b_xi = -np.log(1 - xi)
        ## get first derivative of b_xi with respect to xi
        db_dxi = 1 / (1-xi)
        ## get second derivative of b_xi with respect to xi
        d2b_d2xi = (2-xi) / ((1-xi)**2)
        
        ## get matrix W_inv and Y_til
        W_inv = np.diag(((dr_dxi**2)*(dh_deta**2)*((xi-T_y)**2)).squeeze(axis=1))
        Y_til = 1 / (dr_dxi * dh_deta * (xi-T_y))
        
        ## calculate WLS estimator
        WLS = np.linalg.solve(np.dot(X.T, W_inv).dot(X), np.eye(m)).dot(X.T).dot(W_inv).dot(Y_til)  
                
        ## get the updated Beta
        Beta = Beta - WLS
        del Eta, h_eta, dh_deta, xi, r_xi, dr_dxi, d2r_d2xi, b_xi, db_dxi, d2b_d2xi, WLS
        
        ## calculate the updated cost J
        Eta = np.dot(X, Beta)
        h_eta = probit_activation(Eta)
        xi = h_eta
        r_xi = np.log(xi / (1-xi)) 
        b_xi = -np.log(1 - xi)
        J = np.sum(b_xi - (T_y * r_xi))
        df_algo_results.loc[i+1, 'J'] = J
        del Eta, h_eta, xi, b_xi, r_xi, J
    
    return(Beta, df_algo_results)

In [20]:
#########################
## Logistic Regression ##
#########################
df_logistic, model_logistic = simulate_df(n=1000, glm_type='logistic')
print(model_logistic.summary())
## Newton Raphson / Fisher Scoring
Beta_Logistic_NR, df_Logistic_cost_NR = NewtonRaphson_FisherScoring_Canonical(df=df_logistic, glm_type='logistic', seed=123456)
## IRLS
Beta_Logistic_IRLS, df_Logistic_cost_IRLS = IRLS_Canonical(df=df_logistic, glm_type='logistic', seed=123456)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      993
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -540.88
Date:                Mon, 03 May 2021   Deviance:                       1081.8
Time:                        16:06:16   Pearson chi2:                     970.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1731      0.186     -0.932      0.3

In [21]:
print(Beta_Logistic_NR)

[[-0.17312538]
 [-1.53654037]
 [ 1.69232727]
 [ 0.34965883]
 [ 1.30410149]
 [-1.09423892]
 [ 1.1910266 ]]


In [22]:
print(Beta_Logistic_IRLS)

[[-0.17312538]
 [-1.53654037]
 [ 1.69232727]
 [ 0.34965883]
 [ 1.30410149]
 [-1.09423892]
 [ 1.1910266 ]]


In [23]:
########################
## Poisson Regression ##
########################
df_poisson, model_poisson = simulate_df(n=1000, glm_type='poisson')
print(model_poisson.summary())
## Newton Raphson / Fisher Scoring
Beta_Poisson_NR, df_Poisson_cost_NR = NewtonRaphson_FisherScoring_Canonical(df=df_poisson, glm_type='poisson', seed=123456)
## IRLS
Beta_Poisson_IRLS, df_Poisson_cost_IRLS = IRLS_Canonical(df=df_poisson, glm_type='poisson', seed=123456)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      993
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1605.6
Date:                Mon, 03 May 2021   Deviance:                       1026.7
Time:                        16:06:46   Pearson chi2:                     989.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4635      0.092     -5.065      0.0

In [24]:
print(Beta_Poisson_NR)

[[-0.46354375]
 [-1.5960801 ]
 [ 1.95764143]
 [ 0.41513364]
 [ 1.31072289]
 [-1.19784871]
 [ 1.06158341]]


In [25]:
print(Beta_Poisson_IRLS)

[[-0.46354375]
 [-1.5960801 ]
 [ 1.95764143]
 [ 0.41513364]
 [ 1.31072289]
 [-1.19784871]
 [ 1.06158341]]


In [26]:
#######################
## Probit Regression ##
#######################
df_probit, model_probit = simulate_df(n=1000, glm_type='probit')
print(model_probit.summary())
## Newton Raphson
Beta_Probit_NR, df_Probit_cost_NR = NewtonRaphson_NonCanonical(df=df_probit, seed=123456)
## Fisher Scoring
Beta_Probit_FS, df_Probit_cost_FS = FisherScoring_NonCanonical(df=df_probit, seed=123456)
## IRLS
Beta_Probit_IRLS, df_Probit_cost_IRLS = IRLS_NonCanonical(df=df_probit, seed=123456)

Optimization terminated successfully.
         Current function value: 0.400630
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 1000
Model:                         Probit   Df Residuals:                      993
Method:                           MLE   Df Model:                            6
Date:                Mon, 03 May 2021   Pseudo R-squ.:                  0.3910
Time:                        16:07:22   Log-Likelihood:                -400.63
converged:                       True   LL-Null:                       -657.88
Covariance Type:            nonrobust   LLR p-value:                6.304e-108
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
L0            -0.2544      0.126     -2.024      0.043      -0.501      -0.008
L1            -1.6821      0.

In [27]:
print(Beta_Probit_NR)

[[-0.25437885]
 [-1.68207182]
 [ 1.78983894]
 [ 0.42038955]
 [ 1.21294796]
 [-1.16274506]
 [ 1.15957839]]


In [28]:
print(Beta_Probit_FS)

[[-0.2543798 ]
 [-1.68207257]
 [ 1.78984032]
 [ 0.42038961]
 [ 1.2129496 ]
 [-1.16274281]
 [ 1.15957877]]


In [29]:
print(Beta_Probit_IRLS)

[[-0.2543798 ]
 [-1.68207257]
 [ 1.78984032]
 [ 0.42038961]
 [ 1.2129496 ]
 [-1.16274281]
 [ 1.15957877]]


In [32]:
import jovian

In [33]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/atrothman/glms-newton-raphson-fisher-scoring-irls-simulations


'https://jovian.ai/atrothman/glms-newton-raphson-fisher-scoring-irls-simulations'